# Konopka Bartosz - grupa 2

# Wstęp


W poniższych zadaniach wykorzystaliśmy i badaliśmy sieci wielowarstwowe, korzystając z modelu MLPClassifier. Korzystając z odpowiednich zbiorów danych przetestowaliśmy zależności takie jak struktura sieci, skuteczność sieci w wielu problemach oraz wpływ różnych architektur sieci, funkcji aktywacji, ilości epok uczenia i algorytmów uczenia na wynik zadania.

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn import preprocessing
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# ZAD 1


## Kod

Treść:

Proszę pobrać plik medicine.txt, zawierający wyniki analizy nowego leku. W dwóch pierwszych kolumnach znajduje się stężenie dwóch składników w próbce krwi, w trzeciej - informacja o tym, czy lek zadziałał. Dane nie są znormalizowane. Proszę znormalizować dane, podzielić je na zbiór uczący i testujący w proporcjach 80-20 (należy pamiętać o proporcjach klas), zaproponować wielowarstwową sieć neuronową i zbadać jej skuteczność dla różnych ilości warstw i neuronów w tych warstwach. Proszę narysować w jaki sposób sieć dokonała podziału w zbiorze dla kilku sieci (zarówno tych z dobrymi, jak i złymi wynikami) oraz jak wygląda poprawny podział zbioru. Proszę również przedstawić wyniki dla 5-8 różnych struktur sieci, wraz z oceną, która z nich najlepiej poradziła sobie z zadaniem klasyfikacji.

In [ ]:
data_medicine = pd.read_csv('medicine.txt', sep = ',', header = None)
data_medicine = data_medicine.drop(0)

scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(data_medicine.iloc[:,0:2])
Y = data_medicine.iloc[:,2]
Y = Y.to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size=0.2)

In [ ]:
layers = [(1,1), (5,5), (10,5), (10,10), (50,20), (10,5,2), (50,20,5), (100,50,20)]

for i in layers:
  model = MLPClassifier(hidden_layer_sizes = i, max_iter=2000)
  model.fit(X_train, Y_train)
  display = DecisionBoundaryDisplay.from_estimator(model, X_train, cmap='viridis')
  plt.scatter(X_train[:,0],X_train[:,1], c=Y_train)

  predicted_labels = model.predict(X_test)
  print("score: ", model.score(X_train,Y_train))

  plt.show()


## Wnioski

W eksperymencie z klasyfikacją nowego leku za pomocą wielowarstwowej sieci neuronowej, różne konfiguracje sieci przynosiły zróżnicowane wyniki skuteczności. W przypadku prostszych struktur, składających się z jednej lub dwóch warstw o mniejszej liczbie neuronów, skuteczność była niższa. Jednak złożone modele z większą liczbą warstw i neuronów wykazywały tendencję do poprawy skuteczności. W niektórych przypadkach zwiększenie złożoności sieci nie przynosiło istotnej poprawy wyników. Najlepsze rezultaty uzyskano przy modelu z trzema warstwami (100, 50, 20), osiągając accuracy na poziomie 0.915.

# ZAD 2


## Kod

Treść:

Proszę pobrać zbiór ręcznie pisanych cyfr z https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits (można to zrobić funkcją datasets.load_digits() w sklearnie). Proszę sprawdzić skuteczność klasyfikacji na tym zbiorze za pomocą wielowarstwowej sieci neuronowej.

In [ ]:
data_digits = datasets.load_digits()

X = data_digits.data
Y = data_digits.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size=0.2)

In [ ]:
model = MLPClassifier(solver = "sgd", hidden_layer_sizes=(100,50), max_iter = 1000, tol = 0.001, activation = 'identity')

model.fit(X_train,Y_train)
predicted_labels = model.predict(X_train)

print(confusion_matrix(Y_train, predicted_labels))
print(model.score(X_test,Y_test))

## Wnioski

Skuteczność klasyfikacji na zbiorze ręcznie pisanych cyfr za pomocą wielowarstwowej sieci neuronowej wynosiła 0.9638888888888889 na zbiorze testowym. Jest to bardzo wysoki wynik, co sugeruje, że model dobrze radzi sobie z zadaniem rozpoznawania cyfr na podstawie obrazów.

# ZAD 3

## Kod

Treść:

Proszę sprawdzić, jak zmieni się poprawność klasyfikacji na zbiorze ręcznie pisanych cyfr dla różnych architektur sieci, funkcji aktywacji, ilości epok uczenia i algorytmów uczenia. Proszę zbadać wpływ współczynnika uczenia (learning_rate) podczas używania algorytmu SGD. Dla najciekawszych przykładów proszę wypisać macierze pomyłek.

In [ ]:
data_digits = datasets.load_digits()

X = data_digits.data
Y = data_digits.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size=0.2)

In [8]:
architectures = [(10, 5,5,5), (10, 5,5,5,5), (10, 5,5,5,5,5)]
activation_functions = ['relu', 'logistic', 'tanh']
epochs = [500, 1000, 2000]
learning_rates = [0.1, 0.01, 0.001]
optimizers = ['sgd']
i = 0

for architecture in architectures:
    for activation_function in activation_functions:
        for epoch in epochs:
            for learning_rate in learning_rates:
                for optimizer in optimizers:
                    model = MLPClassifier(hidden_layer_sizes=architecture, activation=activation_function, max_iter=epoch, learning_rate_init=learning_rate, solver=optimizer)
                    model.fit(X_train, Y_train)

                    test_accuracy = model.score(X_test, Y_test)
                    print(f"Number: {i}, Architecture: {architecture}, Activation: {activation_function}, Epochs: {epoch}, Learning Rate: {learning_rate}, Optimizer: {optimizer}")
                    print(f"Test Accuracy: {test_accuracy}")

                    if i % 10 == 0:
                      Y_pred = model.predict(X_test)
                      matrix = confusion_matrix(Y_test, Y_pred)
                      print("Confusion Matrix:")
                      print(matrix)
                    print()

                    i+=1



Number: 79, Architecture: (10, 5, 5, 5, 5, 5), Activation: tanh, Epochs: 2000, Learning Rate: 0.01, Optimizer: sgd
Test Accuracy: 0.7638888888888888

Number: 80, Architecture: (10, 5, 5, 5, 5, 5), Activation: tanh, Epochs: 2000, Learning Rate: 0.001, Optimizer: sgd
Test Accuracy: 0.6527777777777778
Confusion Matrix:
[[34  0  1  0  0  1  0  0  0  0]
 [ 0 29  4  2  1  0  0  0  0  0]
 [ 0  6 29  0  0  0  0  0  0  0]
 [ 2  0  0 33  1  0  0  0  1  0]
 [ 0  0  0  1  7  0 26  0  2  0]
 [ 5  1  3  3  0 25  0  0  0  0]
 [ 1  0  0  0  2  0 32  1  0  0]
 [ 0  0  0  2  0  1  2 31  0  0]
 [ 0 15  0 10  0  0  1  0  9  0]
 [14  1  1  4  0  4  0  0  6  6]]



/home/skolanko/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


## Wnioski

W naszych badaniach różnych architektur sieci neuronowych zauważono, że dodawanie dodatkowych warstw i neuronów może potencjalnie poprawić skuteczność klasyfikacji. Jednakże, nie zawsze większa złożoność przekładała się na lepsze wyniki. Przetestowano trzy funkcje aktywacji: relu, logistic i tanh, z czego relu często wydawała się dawać najlepsze wyniki, choć ostatecznie wybór funkcji aktywacji zależy od specyfiki danych. Dodatkowo, badano wpływ liczby epok uczenia (500, 1000, 2000) i współczynnika uczenia (0.1, 0.01, 0.001) na skuteczność klasyfikacji. Wyniki sugerują, że zwiększenie liczby epok może poprawić skuteczność, ale tylko do pewnego punktu, po którym dalsze zwiększanie nie przynosi dodatkowych korzyści lub prowadzi do nadmiernej złożoności modelu. Podobnie, wybór odpowiedniego współczynnika uczenia jest kluczowy, zbyt duży prowadzi do nadmiernej zmienności, a zbyt mały do wolnego uczenia się modelu.

# ZAD 4

## Kod

Treść:

Proszę pobrać zbiór yeast z UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Yeast). Proszę we własnym zakresie dokonać wstępnej analizy i przygotowania tego zbioru (uwaga, wymagana jest zamiana etykiet tekstowych w ostatniej kolumnie na liczbowe - można je zamienić ręcznie albo przy użyciu takich narzędzi jak https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html, należy jednak pamiętać, że nie musi on ułożyć tych etykiet po kolei). Warto zauważyć, że liczności różnych klas wewnątrz zbioru są bardzo nierówne. Proszę spróbować osiągnąć jak najlepsze wyniki i narysować dla nich macierz pomyłek (dla zbioru uczącego i testującego). Czy trafność na poziomie 0.5 dla takiego zbioru jest dobra? Mogą państwo zbadać też czas wykonywania funkcji fit dla różnych konfiguracji sieci.

In [16]:
url1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data"
data = pd.read_csv(url1, header=None, delimiter=r"\s+")

X = data.iloc[:, 1:-1]
Y = data.iloc[:, -1]

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=42)


In [18]:
model = MLPClassifier(hidden_layer_sizes=(100,50), activation='relu', max_iter=1000, random_state=42)

start_time = time.time()
model.fit(X_train, Y_train)
end_time = time.time()
training_time = end_time - start_time

train_accuracy = model.score(X_train, Y_train)
test_accuracy = model.score(X_test, Y_test)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Training Time: {training_time} seconds")

Y_pred_train = model.predict(X_train)
cm_train = confusion_matrix(Y_train, Y_pred_train)
Y_pred_test = model.predict(X_test)
cm_test = confusion_matrix(Y_test, Y_pred_test)

print("Train confusion matrix: \n", cm_train)
print("Test confusion matrix: \n", cm_test)

ValueError: Found input variables with inconsistent numbers of samples: [1437, 1187]

## Wnioski

Skuteczność klasyfikacji na zbiorze testowym wynosiła około 0.609, a na zbiorze treningowym - około 0.644. Trafność na poziomie 0.5 dla takiego zbioru danych jest niezadowalająca, co sugeruje, że model nie radzi sobie dobrze z zadaniem klasyfikacji w tym przypadku. Analiza macierzy pomyłek pozwala zidentyfikować klasy, które są gorzej klasyfikowane przez model, co może być przydatne w dalszej optymalizacji. Warto zauważyć, że niektóre klasy mają znacznie więcej błędów klasyfikacji niż inne.